Inserción de los datos de Norman en la base de datos de ArangoDB

In [5]:
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_TYPE = "openai"

In [17]:
# Instantiate ArangoDB Database
import json

from adb_cloud_connector import get_temp_credentials
from arango import ArangoClient

con = get_temp_credentials()

db = ArangoClient(hosts=con["url"]).db(
    con["dbName"], con["username"], con["password"], verify=True
)

print(json.dumps(con, indent=2))

Success: reusing cached credentials
{
  "dbName": "TUTj7i7fm55rmp58xz4lw6skg",
  "username": "TUTbu0jwwiqud451fyooi8ww4",
  "password": "TUTue5dopxl90qrvij44492e",
  "hostname": "tutorials.arangodb.cloud",
  "port": 8529,
  "url": "https://tutorials.arangodb.cloud:8529"
}


Se elimina el grafo Norman si ya existe 



In [63]:
if db.has_graph("Norman"):
    db.delete_graph("Norman", drop_collections=True)

Se crean las colecciones necesarias

In [64]:


db.create_graph(
    "Norman",
    edge_definitions=[
        {
            "edge_collection": "is_son_of",
            "from_vertex_collections": ["Persons"],
            "to_vertex_collections": ["Persons"],
        },
        {
            "edge_collection": "is_father_of",
            "from_vertex_collections": ["Persons"],
            "to_vertex_collections": ["Persons"],
        },
    ],
)

normans_family_members = [
    {
        "name": "Norman",
        "_key": "Norman",
        "family_name": "Gonzalez",
        "age": 41,
    },
    {
      "name": "Julia",
      "_key": "Julia",
      "family_name": "Gonzalez",
      "age": 40,
      "relationship_to_Norman": "Wife"
    },
    {
      "name": "Lucas",
      "_key": "Lucas",
      "family_name": "Gonzalez",
      "age": 10,
      "relationship_to_Norman": "Son"
    },
    {
      "name": "Mia",
      "_key": "Mia",
      "family_name": "Gonzalez",
      "age": 8,
      "relationship_to_Norman": "Daughter"
    },
    {
      "name": "Eva",
      "_key": "Eva",
      "family_name": "Gonzalez",
      "age": 8,
      "relationship_to_Norman": "Daughter"
    },
    {
      "name": "Carlos",
      "_key": "Carlos",
      "family_name": "Gonzalez",
      "age": 68,
      "relationship_to_Norman": "Father"
    },
    {
      "name": "Isabel",
      "_key": "Isabel",
      "family_name": "Gonzalez",
      "age": 65,
      "relationship_to_Norman": "Mother"
    },
    {
      "name": "Mario",
      "_key": "Mario",
      "family_name": "Gonzalez",
      "age": 45,
      "relationship_to_Norman": "Brother"
    },
    {
      "name": "Ana",
      "_key": "Ana",
      "family_name": "Martinez",
      "age": 43,
      "relationship_to_Norman": "Sister-in-law (Mario's wife)"
    },
    {
      "name": "Pedro",
      "_key": "Pedro",
      "family_name": "Gonzalez",
      "age": 38,
      "relationship_to_Norman": "Brother"
    },
    {
      "name": "Sofia",
      "_key": "Sofia",
      "family_name": "",
      "age": 42,
      "relationship_to_Norman": ""
    },
    {
      "name": "Rafael",
      "_key": "Rafael",
      "family_name": "Lopez",
      "age": 36
    },
    {
      "name": "Liam",
      "_key": "Liam",
      "family_name": "Lopez",
      "age": 12
    },
    {
      "name": "Olivia",
      "_key": "Olivia",
      "family_name": "",
      "age": 10
    },
    {
      "name": "Ethan",
      "_key": "Ethan",
      "family_name": "",
      "age": 7
    },
    {
      "name": "Amelia",
      "_key": "Amelia",
      "family_name": "",
      "age": 5
    }
    ]

child_of_edges = [
    {"_from": "Persons/Lucas", "_to": "Persons/Norman"},
    {"_from": "Persons/Mia", "_to": "Persons/Norman"},
    {"_from": "Persons/Eva", "_to": "Persons/Norman"},
    {"_from": "Persons/Norman", "_to": "Persons/Carlos"},
]

father_of_edges = [
    {"_from": "Persons/Norman", "_to": "Persons/Lucas"},
    {"_from": "Persons/Norman", "_to": "Persons/Mia"},
    {"_from": "Persons/Norman", "_to": "Persons/Eva"},
    {"_from": "Persons/Carlos", "_to": "Persons/Norman"},
]


db.collection("Persons").import_bulk(normans_family_members)
db.collection("is_son_of").import_bulk(child_of_edges)
db.collection("is_father_of").import_bulk(father_of_edges)

{'error': False,
 'created': 4,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

In [65]:
# Instantiate the ArangoDB-LangChain Graph
from langchain_community.graphs import ArangoGraph
graph = ArangoGraph(db)

In [66]:
graph.set_schema()

In [67]:
graph

In [68]:
# We can now view the generated schema

import json

print(json.dumps(graph.schema, indent=4))

{
    "Graph Schema": [
        {
            "graph_name": "Norman",
            "edge_definitions": [
                {
                    "edge_collection": "is_father_of",
                    "from_vertex_collections": [
                        "Persons"
                    ],
                    "to_vertex_collections": [
                        "Persons"
                    ]
                },
                {
                    "edge_collection": "is_son_of",
                    "from_vertex_collections": [
                        "Persons"
                    ],
                    "to_vertex_collections": [
                        "Persons"
                    ]
                }
            ]
        }
    ],
    "Collection Schema": [
        {
            "collection_name": "is_father_of",
            "collection_type": "edge",
            "edge_properties": [
                {
                    "name": "_key",
                    "type": "str"
                },
    

In [69]:
from langchain.chains import ArangoGraphQAChain
from langchain_openai import ChatOpenAI

chain = ArangoGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, allow_dangerous_requests=True
)

Hacemos una pregunta desde un tercero ajeno a la persona de estudio en lenguaje natural

In [71]:
chain.run("de quien es hijo Norman?")




> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, is_father_of
FOR v, e, p IN 1..1 OUTBOUND 'Persons/Norman' is_father_of
RETURN v

AQL Result:
[{'_key': 'Eva', '_id': 'Persons/Eva', '_rev': '_iho5hiW--C', 'name': 'Eva', 'family_name': 'Gonzalez', 'age': 8, 'relationship_to_Norman': 'Daughter'}, {'_key': 'Mia', '_id': 'Persons/Mia', '_rev': '_iho5hiW--B', 'name': 'Mia', 'family_name': 'Gonzalez', 'age': 8, 'relationship_to_Norman': 'Daughter'}, {'_key': 'Lucas', '_id': 'Persons/Lucas', '_rev': '_iho5hiW--A', 'name': 'Lucas', 'family_name': 'Gonzalez', 'age': 10, 'relationship_to_Norman': 'Son'}]

> Finished chain.


'Norman tiene varios hijos. Entre ellos se encuentran Eva, Mia y Lucas. Lucas es el hijo de Norman.'

In [40]:
chain.run("quienes son los hijos de Norman?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
FOR v, e IN 1..1 OUTBOUND 'Persons/Norman' ChildOf
    RETURN v

AQL Result:
[{'_key': 'Carlos', '_id': 'Persons/Carlos', '_rev': '_ihop_Sa--E', 'name': 'Carlos', 'family_name': 'Gonzalez', 'age': 68, 'relationship_to_Norman': 'Father'}]

> Finished chain.


"Summary:\nThe children of Norman are Carlos. Carlos is 68 years old and is identified as Norman's father in the database."

In [27]:
chain.run("add a new son of Norman named Xavier")





> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
LET norman = (
    FOR person IN Persons
    FILTER person.name == 'Norman'
    RETURN person
)[0]

LET xavier = {
    "_key": "Xavier",
    "name": "Xavier",
    "family_name": "Gonzalez",
    "age": 0
}

INSERT xavier INTO Persons

LET edge = {
    "_from": norman._id,
    "_to": CONCAT('Persons/', xavier._key)
}

INSERT edge INTO ChildOf

AQL Result:
[]

> Finished chain.


'A new son named Xavier has been successfully added to Norman in the database.'

In [28]:
chain.run("Norman tiene un hijo Xavier?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
FOR v, e, p IN 1..1 OUTBOUND 'Persons/Norman' ChildOf
FILTER v.name == 'Xavier'
RETURN v

AQL Result:
[{'_key': 'Xavier', '_id': 'Persons/Xavier', '_rev': '_hux4B6W---', 'name': 'Xavier', 'family_name': 'Gonzalez', 'age': 0}]

> Finished chain.


"Summary:\nYes, Norman has a son named Xavier. According to the database query results, Xavier is related to Norman as his son, with the same family name 'Gonzalez'."